In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import xgboost as xgb

In [2]:
train_df = pd.read_csv("training.csv", sep="\t")
valid_df = pd.read_csv("validation.csv", sep="\t")

In [3]:
train_df.head()

,DepartureDate,DepartureYear,DepartureMonth,DepartureDay,FlightNumber,DepartureAirport,ArrivalAirport,Route,ActualFlightTime,ActualTotalFuel,ActualTOW,FLownPassengers,BagsCount,FlightBagsWeight
0,01/10/2016,2016,10,1,1145,MAN,SXF,MAN-SXF,91,3660,64016,175,61,440
1,01/10/2016,2016,10,1,1160,CTA,FCO,CTA-FCO,68,3280,66138,182,27,350
2,01/10/2016,2016,10,1,1183,LGW,SNN,LGW-SNN,64,2720,58447,127,(null),(null)
3,01/10/2016,2016,10,1,1220,ATH,CHQ,ATH-CHQ,35,1682,60587,163,12,150
4,01/10/2016,2016,10,1,1225,CHQ,ATH,CHQ-ATH,34,1877,63090,178,37,490


In [4]:
valid_df.head()

,DepartureDate,DepartureYear,DepartureMonth,DepartureDay,FlightNumber,DepartureAirport,ArrivalAirport,Route,ActualFlightTime,ActualTotalFuel,FLownPassengers,BagsCount,FlightBagsWeight
0,01/11/2016,2016,11,1,6003,MRS,BES,MRS-BES,84,3630,185,43,560
1,01/11/2016,2016,11,1,6004,BES,MRS,BES-MRS,86,3690,188,(null),(null)
2,01/11/2016,2016,11,1,6542,MRS,STN,MRS-STN,100,4390,168,55,720
3,01/11/2016,2016,11,1,6541,STN,MRS,STN-MRS,100,4690,185,35,460
4,01/11/2016,2016,11,1,5542,STN,REU,STN-REU,112,4900,167,31,400


In [5]:
print(f"Training data: {len(train_df)}")
print(f"Validation data: {len(valid_df)}")

Training data: 29731
Validation data: 1878


In [6]:
print(train_df.columns)

Index(['DepartureDate', 'DepartureYear', 'DepartureMonth', 'DepartureDay',
       'FlightNumber', 'DepartureAirport', 'ArrivalAirport', 'Route',
       'ActualFlightTime', 'ActualTotalFuel', 'ActualTOW', 'FLownPassengers',
       'BagsCount', 'FlightBagsWeight'],
      dtype='object')


In [7]:
train_df.rename(columns={'FLownPassengers': 'FlownPassengers'}, inplace=True)

In [8]:
print(valid_df.columns)

Index(['DepartureDate', 'DepartureYear', 'DepartureMonth', 'DepartureDay',
       'FlightNumber', 'DepartureAirport', 'ArrivalAirport', 'Route',
       'ActualFlightTime', 'ActualTotalFuel', 'FLownPassengers', 'BagsCount',
       'FlightBagsWeight'],
      dtype='object')


In [9]:
valid_df.rename(columns={'FLownPassengers': 'FlownPassengers'}, inplace=True)

In [10]:
print(train_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29731 entries, 0 to 29730
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   DepartureDate     29731 non-null  object
 1   DepartureYear     29731 non-null  int64 
 2   DepartureMonth    29731 non-null  int64 
 3   DepartureDay      29731 non-null  int64 
 4   FlightNumber      29731 non-null  int64 
 5   DepartureAirport  29731 non-null  object
 6   ArrivalAirport    29731 non-null  object
 7   Route             29731 non-null  object
 8   ActualFlightTime  29731 non-null  int64 
 9   ActualTotalFuel   29731 non-null  int64 
 10  ActualTOW         29731 non-null  object
 11  FlownPassengers   29731 non-null  object
 12  BagsCount         29731 non-null  object
 13  FlightBagsWeight  29731 non-null  object
dtypes: int64(6), object(8)
memory usage: 3.2+ MB
None


In [11]:
numeric_columns = ['ActualFlightTime', 'ActualTotalFuel', 'ActualTOW',
                   'FlownPassengers', 'BagsCount', 'FlightBagsWeight']

for col in numeric_columns:
    if col in train_df.columns:
        train_df[col] = pd.to_numeric(train_df[col], errors='coerce')

In [12]:
print(train_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29731 entries, 0 to 29730
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   DepartureDate     29731 non-null  object 
 1   DepartureYear     29731 non-null  int64  
 2   DepartureMonth    29731 non-null  int64  
 3   DepartureDay      29731 non-null  int64  
 4   FlightNumber      29731 non-null  int64  
 5   DepartureAirport  29731 non-null  object 
 6   ArrivalAirport    29731 non-null  object 
 7   Route             29731 non-null  object 
 8   ActualFlightTime  29731 non-null  int64  
 9   ActualTotalFuel   29731 non-null  int64  
 10  ActualTOW         29298 non-null  float64
 11  FlownPassengers   29636 non-null  float64
 12  BagsCount         27447 non-null  float64
 13  FlightBagsWeight  27253 non-null  float64
dtypes: float64(4), int64(6), object(4)
memory usage: 3.2+ MB
None


In [13]:
train_df["DepartureDate"] = pd.to_datetime(train_df["DepartureDate"], format="%d/%m/%Y")
train_df["DayOfWeek"] = train_df["DepartureDate"].dt.weekday

train_df["FuelPerMinute"] = train_df["ActualTotalFuel"] / train_df["ActualFlightTime"]
train_df['FuelPerPassenger'] = train_df['ActualTotalFuel'] / (train_df['FlownPassengers'])


train_df["PassengersPerBag"] = train_df["FlownPassengers"] / train_df["BagsCount"]
train_df['AvgBagWeight'] = train_df['FlightBagsWeight'] / (train_df['BagsCount'])


train_df['IsWeekend'] = train_df['DayOfWeek'].isin([5, 6]).astype(int)

In [14]:
print(train_df['IsWeekend'].value_counts())

IsWeekend
0    19950
1     9781
Name: count, dtype: int64


In [15]:
train_df.head()

,DepartureDate,DepartureYear,DepartureMonth,DepartureDay,FlightNumber,DepartureAirport,ArrivalAirport,Route,ActualFlightTime,ActualTotalFuel,ActualTOW,FlownPassengers,BagsCount,FlightBagsWeight,DayOfWeek,FuelPerMinute,FuelPerPassenger,PassengersPerBag,AvgBagWeight,IsWeekend
0,2016-10-01,2016,10,1,1145,MAN,SXF,MAN-SXF,91,3660,64016.0,175.0,61.0,440.0,5,40.219780,20.914286,2.868852,7.213115,1
1,2016-10-01,2016,10,1,1160,CTA,FCO,CTA-FCO,68,3280,66138.0,182.0,27.0,350.0,5,48.235294,18.021978,6.740741,12.962963,1
2,2016-10-01,2016,10,1,1183,LGW,SNN,LGW-SNN,64,2720,58447.0,127.0,NaN,NaN,5,42.500000,21.417323,NaN,NaN,1
3,2016-10-01,2016,10,1,1220,ATH,CHQ,ATH-CHQ,35,1682,60587.0,163.0,12.0,150.0,5,48.057143,10.319018,13.583333,12.500000,1
4,2016-10-01,2016,10,1,1225,CHQ,ATH,CHQ-ATH,34,1877,63090.0,178.0,37.0,490.0,5,55.205882,10.544944,4.810811,13.243243,1


In [16]:
n_missing_rows = train_df.isnull().any(axis=1).sum()
n_complete_rows = train_df.shape[0] - n_missing_rows

print(f"Missing rows: {n_missing_rows}")
print(f"Full rows: {n_complete_rows}")

Missing rows: 2931
Full rows: 26800


In [17]:
print(train_df.isna().sum())

DepartureDate          0
DepartureYear          0
DepartureMonth         0
DepartureDay           0
FlightNumber           0
DepartureAirport       0
ArrivalAirport         0
Route                  0
ActualFlightTime       0
ActualTotalFuel        0
ActualTOW            433
FlownPassengers       95
BagsCount           2284
FlightBagsWeight    2478
DayOfWeek              0
FuelPerMinute          0
FuelPerPassenger      95
PassengersPerBag    2284
AvgBagWeight        2589
IsWeekend              0
dtype: int64


In [18]:
train_df = train_df.dropna()

In [19]:
print(train_df.isna().sum())

DepartureDate       0
DepartureYear       0
DepartureMonth      0
DepartureDay        0
FlightNumber        0
DepartureAirport    0
ArrivalAirport      0
Route               0
ActualFlightTime    0
ActualTotalFuel     0
ActualTOW           0
FlownPassengers     0
BagsCount           0
FlightBagsWeight    0
DayOfWeek           0
FuelPerMinute       0
FuelPerPassenger    0
PassengersPerBag    0
AvgBagWeight        0
IsWeekend           0
dtype: int64


In [20]:
num_duplicates = train_df.duplicated().sum()

if num_duplicates > 0:
    print(f"Found {num_duplicates} duplicates:")
    display(train_df[train_df.duplicated()].head())
else:
    print("No duplicates.")

No duplicates.


In [21]:
print(train_df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 26800 entries, 0 to 29730
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   DepartureDate     26800 non-null  datetime64[ns]
 1   DepartureYear     26800 non-null  int64         
 2   DepartureMonth    26800 non-null  int64         
 3   DepartureDay      26800 non-null  int64         
 4   FlightNumber      26800 non-null  int64         
 5   DepartureAirport  26800 non-null  object        
 6   ArrivalAirport    26800 non-null  object        
 7   Route             26800 non-null  object        
 8   ActualFlightTime  26800 non-null  int64         
 9   ActualTotalFuel   26800 non-null  int64         
 10  ActualTOW         26800 non-null  float64       
 11  FlownPassengers   26800 non-null  float64       
 12  BagsCount         26800 non-null  float64       
 13  FlightBagsWeight  26800 non-null  float64       
 14  DayOfWeek         26800 non

In [22]:
train_df.drop(columns=["DepartureDate", "FlightNumber"], inplace=True)

In [23]:
print(train_df.columns)

Index(['DepartureYear', 'DepartureMonth', 'DepartureDay', 'DepartureAirport',
       'ArrivalAirport', 'Route', 'ActualFlightTime', 'ActualTotalFuel',
       'ActualTOW', 'FlownPassengers', 'BagsCount', 'FlightBagsWeight',
       'DayOfWeek', 'FuelPerMinute', 'FuelPerPassenger', 'PassengersPerBag',
       'AvgBagWeight', 'IsWeekend'],
      dtype='object')


In [24]:
all_features = ['DepartureYear', 'DepartureMonth', 'DepartureDay',
       'DepartureAirport', 'ArrivalAirport', 'Route', 'ActualFlightTime',
       'ActualTotalFuel', 'FlownPassengers', 'BagsCount',
       'FlightBagsWeight', 'DayOfWeek', 'FuelPerMinute', 'FuelPerPassenger',
       'PassengersPerBag', 'AvgBagWeight', 'IsWeekend']

X = train_df[all_features].copy()
y = train_df['ActualTOW']

In [25]:
for col in X.select_dtypes(include='object').columns:
    X[col] = X[col].astype('category')

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
import optuna


def objective(trial):

    selected_features = [f for f in all_features if trial.suggest_categorical(f'use_{f}', [True, False])]
    if len(selected_features) == 0:
        return float('inf') 

    X_selected = X[selected_features]

    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 5),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 5),
        'tree_method': 'hist',
        'enable_categorical': True,
        'random_state': 42,
    }

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    y_bins = pd.qcut(y, q=10, duplicates='drop', labels=False)
    mse_scores = []

    for train_idx, val_idx in skf.split(X_selected, y_bins):
        X_train, X_val = X_selected.iloc[train_idx], X_selected.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        model = XGBRegressor(**params)
        model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            verbose=False
        )
        y_pred = model.predict(X_val)
        mse_scores.append(mean_squared_error(y_val, y_pred))

    return np.mean(mse_scores)

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=400)

print("Best parameters:")
print(study.best_params)
print(f"Best MSE: {study.best_value:.2f}")

best_features = [f for f in all_features if study.best_params.get(f'use_' + f, False)]
print("Features:")
print(best_features)

[I 2025-05-29 12:59:29,991] A new study created in memory with name: no-name-3153e7ca-831d-4b6b-a769-307fcb0b87bf
[I 2025-05-29 13:00:55,010] Trial 0 finished with value: 1465799.2352596384 and parameters: {'use_DepartureYear': True, 'use_DepartureMonth': False, 'use_DepartureDay': True, 'use_DepartureAirport': True, 'use_ArrivalAirport': True, 'use_Route': True, 'use_ActualFlightTime': True, 'use_ActualTotalFuel': False, 'use_FlownPassengers': False, 'use_BagsCount': False, 'use_FlightBagsWeight': True, 'use_DayOfWeek': False, 'use_FuelPerMinute': False, 'use_FuelPerPassenger': False, 'use_PassengersPerBag': True, 'use_AvgBagWeight': True, 'use_IsWeekend': False, 'n_estimators': 414, 'max_depth': 9, 'learning_rate': 0.06969161685456265, 'subsample': 0.8916623345919852, 'colsample_bytree': 0.9374584847963141, 'gamma': 1.9219895307328434, 'reg_alpha': 3.777290754827533, 'reg_lambda': 1.1533615042484207}. Best is trial 0 with value: 1465799.2352596384.
[I 2025-05-29 13:01:23,440] Trial 1

Best parameters:
{'use_DepartureYear': True, 'use_DepartureMonth': True, 'use_DepartureDay': True, 'use_DepartureAirport': True, 'use_ArrivalAirport': True, 'use_Route': False, 'use_ActualFlightTime': True, 'use_ActualTotalFuel': True, 'use_FlownPassengers': True, 'use_BagsCount': False, 'use_FlightBagsWeight': True, 'use_DayOfWeek': True, 'use_FuelPerMinute': True, 'use_FuelPerPassenger': False, 'use_PassengersPerBag': False, 'use_AvgBagWeight': True, 'use_IsWeekend': True, 'n_estimators': 470, 'max_depth': 4, 'learning_rate': 0.09694081261961722, 'subsample': 0.9214753371833588, 'colsample_bytree': 0.6674115091571178, 'gamma': 0.8418003889366719, 'reg_alpha': 1.5778679003672926, 'reg_lambda': 2.1406100860160375}
Best MSE: 465691.77
Features:
['DepartureYear', 'DepartureMonth', 'DepartureDay', 'DepartureAirport', 'ArrivalAirport', 'ActualFlightTime', 'ActualTotalFuel', 'FlownPassengers', 'FlightBagsWeight', 'DayOfWeek', 'FuelPerMinute', 'AvgBagWeight', 'IsWeekend']


In [ ]:
from xgboost import XGBRegressor
import joblib

best_features = [f for f in all_features if study.best_params.get(f'use_' + f, False)]
best_params = {k: v for k, v in study.best_params.items() if not k.startswith('use_')}

final_model = XGBRegressor(
    **best_params,
    tree_method='hist',
    enable_categorical=True,
    random_state=42
)

X_final = X[best_features]
final_model.fit(X_final, y)

joblib.dump(final_model, 'best_xgb_model.pkl')
print("Saved as 'best_xgb_model.pkl'")


Saved as 'best_xgb_model.pkl'


In [26]:
print(valid_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1878 entries, 0 to 1877
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   DepartureDate     1878 non-null   object
 1   DepartureYear     1878 non-null   int64 
 2   DepartureMonth    1878 non-null   int64 
 3   DepartureDay      1878 non-null   int64 
 4   FlightNumber      1878 non-null   int64 
 5   DepartureAirport  1878 non-null   object
 6   ArrivalAirport    1878 non-null   object
 7   Route             1878 non-null   object
 8   ActualFlightTime  1878 non-null   int64 
 9   ActualTotalFuel   1878 non-null   int64 
 10  FlownPassengers   1878 non-null   object
 11  BagsCount         1878 non-null   object
 12  FlightBagsWeight  1878 non-null   object
dtypes: int64(6), object(7)
memory usage: 190.9+ KB
None


In [29]:
missing = [col for col in best_features if col not in valid_df.columns]
assert not missing, f"Missing columns in valid_df: {missing}"

AssertionError: Missing columns in valid_df: ['DayOfWeek', 'FuelPerMinute', 'AvgBagWeight', 'IsWeekend']

In [31]:
numeric_columns = ['ActualFlightTime', 'ActualTotalFuel',
                   'FlownPassengers', 'BagsCount', 'FlightBagsWeight']

for col in numeric_columns:
    if col in valid_df.columns:
        valid_df[col] = pd.to_numeric(valid_df[col], errors='coerce')

In [32]:
valid_df["DepartureDate"] = pd.to_datetime(valid_df["DepartureDate"], format="%d/%m/%Y")
valid_df["DayOfWeek"] = valid_df["DepartureDate"].dt.weekday

valid_df["FuelPerMinute"] = valid_df["ActualTotalFuel"] / valid_df["ActualFlightTime"]
valid_df['FuelPerPassenger'] = valid_df['ActualTotalFuel'] / (valid_df['FlownPassengers'])


valid_df["PassengersPerBag"] = valid_df["FlownPassengers"] / valid_df["BagsCount"]
valid_df['AvgBagWeight'] = valid_df['FlightBagsWeight'] / (valid_df['BagsCount'])


valid_df['IsWeekend'] = valid_df['DayOfWeek'].isin([5, 6]).astype(int)

In [33]:
for col in valid_df.select_dtypes(include='object').columns:
    valid_df[col] = valid_df[col].astype('category')

In [ ]:
valid_df = valid_df[best_features].copy()

In [35]:
valid_df['PredictedTOW'] = final_model.predict(valid_df)

In [36]:
valid_df.to_csv("predicted.csv", index=False)